# **SENTIMENT ANALYSIS WITH TENSORFLOW**

<img src="https://inkygoodness.com/wp-content/uploads/2018/01/human_emotions-IG-vanessa.jpg" width=600>

# Stage 1: Understand the basic of Tokenizer and Padding of Tensorflow

## Tokenization


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Our sentences
sentences =[
            'i love my dog',
            'I, love my cat'
]

**num_word** is the maximum number of words we gonna keep. It is ok because we have only two sentences now, but imagine we got hundreds of books to tokenize, with 10 millions unique words and we just want 100 words in all of that if num_word=100.

In [ ]:
tokenizer = Tokenizer(num_words=100)

fit_on_texts is the method for tokenizer to learn how many unique words in our collection and what kind of frequency of the words appearing:

In [ ]:
tokenizer.fit_on_texts(sentences)

We can check out how many unique words in the text. 

**Question:** Can you guess why some specific words got small index number and some words got high index number?

In [ ]:
word_index = tokenizer.word_index
print(word_index)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'cat': 5}


**The tokenizer is smart enough to catch some exceptions like this! Note that dog with "!"**

In [ ]:
# Our sentences
sentences =[
            'i love my dog',
            'I, love my <cat>',
            'You love my dog!'
]

In [ ]:
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'love': 1, 'my': 2, 'i': 3, 'dog': 4, 'cat': 5, 'you': 6}


More info about Tokenization: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

You can see how words can be tokenized and tools in Tensorflow can handle that for you.

Now your words are represented by numbers like this then you need to represent your sentences by sequences of numbers in the correct order. 

## Turning sentences into numbers

Time to create sequences from sentences!

Let try a different example, this time **these sentences will have different lengths.**

In [ ]:
# Our sentences
sentences =[
            'i love my dog',
            'I, love my cat',
            'You love my dog!',
            'Do you think my dog is amazing?'
]

In [ ]:
tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

In [ ]:
print(word_index)

{'my': 1, 'love': 2, 'dog': 3, 'i': 4, 'you': 5, 'cat': 6, 'do': 7, 'think': 8, 'is': 9, 'amazing': 10}


**text_to_sequences** will create sequences of tokens representing each sentence. Too easy!

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
sequences

[[4, 2, 1, 3], [4, 2, 1, 6], [5, 2, 1, 3], [7, 5, 8, 1, 3, 9, 10]]

You can make sense of the first sentence which is "I love my dog" -> [4, 2, 1, 3]

**What about the words that our model never seen before?**

In this example, we will have **new words "really" and "food"**

In [ ]:
# Try with new setences
test_data=[
           'i really love my dog',
           'my dog loves my food'
]

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_data)
print(test_seq)

[[4, 2, 1, 3], [1, 3, 1]]


In [ ]:
print(word_index)

{'my': 1, 'love': 2, 'dog': 3, 'i': 4, 'you': 5, 'cat': 6, 'do': 7, 'think': 8, 'is': 9, 'amazing': 10}


So you can imagine that you need a really big word index to handle sentences that are not in the training set.

**In order to not lose the length of sequence like above, there is a trick for that!**

We will create a unique word that would never be in any text like **"\<OOV\>"**. Then we can replace words which we never seen before with OOV instead!

In [ ]:
# Let's try again
test_data=[
           'i really love my dog',
           'my dog loves my food'
]

In [ ]:
tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}


In [ ]:
test_seq=tokenizer.texts_to_sequences(test_data)
print(test_seq)

[[5, 1, 3, 2, 4], [2, 4, 1, 2, 1]]


Now, all sequences will have the same length of our original sentences. Pretty neat trick right?

Another problem is that how our model can handle sequences with different sizes/lengths 

Just like when we train images, inputs of the model are needed to be the same size/length.

## Padding sequences

In [ ]:
# Our sentences
sentences =[
            'i love my dog',
            'I, love my cat',
            'You love my dog!',
            'Do you think my dog is amazing?'
]

In [ ]:
tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'my': 2, 'love': 3, 'dog': 4, 'i': 5, 'you': 6, 'cat': 7, 'do': 8, 'think': 9, 'is': 10, 'amazing': 11}


In [ ]:
sequences=tokenizer.texts_to_sequences(sentences)
print(sequences)

[[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]


More Info about pad_sequences: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
padded = pad_sequences(sequences)
print(padded)

[[ 0  0  0  5  3  2  4]
 [ 0  0  0  5  3  2  7]
 [ 0  0  0  6  3  2  4]
 [ 8  6  9  2  4 10 11]]


Nice, so it is padded at the beginining!

What if we want to pad them at the end?

In [ ]:
padded = pad_sequences(sequences, padding='post')
print(padded)

[[ 5  3  2  4  0  0  0]
 [ 5  3  2  7  0  0  0]
 [ 6  3  2  4  0  0  0]
 [ 8  6  9  2  4 10 11]]


We can even set the max_len instead of use the maximum length of the longest sentence. 

If the sentence is too long for our max_len, we can truncate/remove some words to fit it (truncate=post or pre)

In [ ]:
padded = pad_sequences(sequences, padding='post', truncating='post', maxlen=6)
print(padded)

[[ 5  3  2  4  0  0]
 [ 5  3  2  7  0  0]
 [ 6  3  2  4  0  0]
 [ 8  6  9  2  4 10]]


In [ ]:
padded = pad_sequences(sequences, padding='post', truncating='pre', maxlen=6)
print(padded)

[[ 5  3  2  4  0  0]
 [ 5  3  2  7  0  0]
 [ 6  3  2  4  0  0]
 [ 6  9  2  4 10 11]]


Now you know how to tokenize text into numeric values and how to regulaize and pad those text. So the preprocession is done!

Time to train our juicy network model with these representations of sentences to detect if a sentence is positive or negative! However, how can we make sure these numbers be meaningful when it comes to sentiment analysis ? So we need Embedding !

## Embedding Layer

The above is your full embedded matrix. We need to find a way to retrieve correct embedded vector for each word and then for each sentence!

![alt text](https://i.imgur.com/z3qObl7.png)

In [ ]:
# Our sentences
sentences =[
            'i love my dog',
            'I, love my cat',
            'You love my dog!',
            'Do you think my dog is amazing?'
]

tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)

Nice, we got exactly 12 words in vocab!

In [ ]:
word_index

{'<OOV>': 1,
 'amazing': 11,
 'cat': 7,
 'do': 8,
 'dog': 4,
 'i': 5,
 'is': 10,
 'love': 3,
 'my': 2,
 'think': 9,
 'you': 6}

In [ ]:
sequences

[[5, 3, 2, 4], [5, 3, 2, 7], [6, 3, 2, 4], [8, 6, 9, 2, 4, 10, 11]]

In [ ]:
padded = pad_sequences(sequences, padding='post', truncating='pre', maxlen=6)
print(padded)

[[ 5  3  2  4  0  0]
 [ 5  3  2  7  0  0]
 [ 6  3  2  4  0  0]
 [ 6  9  2  4 10 11]]


In [ ]:
padded.shape

(4, 6)

In [ ]:
first_sentence = padded[0]
first_sentence

array([5, 3, 2, 4, 0, 0], dtype=int32)

In [ ]:
word_index

{'<OOV>': 1,
 'amazing': 11,
 'cat': 7,
 'do': 8,
 'dog': 4,
 'i': 5,
 'is': 10,
 'love': 3,
 'my': 2,
 'think': 9,
 'you': 6}

In [ ]:
from tensorflow.keras import layers

In [ ]:
vocab_size = 12 
embedding_dim = 3 # can be represented for good, bad, crazy
embedding_layer = layers.Embedding(vocab_size, embedding_dim)

The shape here make senses:
- We got 12 unique words in our vocab.
- Each word are represented by 3 number (3 dimension vector)

So the size of embedding layer is 12 by 3 ;)

In [ ]:
embedding_layer.input_dim, embedding_layer.output_dim

(12, 3)

Let's try to get all embedding vectors from the layer!

In [ ]:
result = embedding_layer(tf.constant([0,1,2,3,4,5,6,7,8,9,10,11]))
result.numpy()

array([[-2.1744519e-05,  3.8744774e-02,  4.5454454e-02],
       [ 3.7292365e-02, -4.5226324e-02, -2.5365019e-02],
       [-5.4663531e-03,  4.1066814e-02, -3.5211097e-02],
       [-3.7309039e-02,  3.7818067e-03,  1.9867215e-02],
       [-8.1157684e-03,  1.8775824e-02,  1.3189439e-02],
       [-3.1944454e-02, -2.0747567e-02, -4.5926087e-03],
       [ 3.6478329e-02,  1.1744276e-03, -7.2725639e-03],
       [ 3.3895124e-02,  1.9534323e-02,  2.2506867e-02],
       [-4.7945656e-02,  5.8999285e-03,  3.5151947e-02],
       [ 3.3130024e-02, -1.9323707e-02,  3.8050737e-02],
       [-3.0018687e-03,  1.9134048e-02,  1.2921009e-02],
       [-2.5133276e-02,  2.6607402e-03,  1.8285479e-02]], dtype=float32)

In [ ]:
word_index

{'<OOV>': 1,
 'amazing': 11,
 'cat': 7,
 'do': 8,
 'dog': 4,
 'i': 5,
 'is': 10,
 'love': 3,
 'my': 2,
 'think': 9,
 'you': 6}

In [ ]:
result = embedding_layer(tf.constant([0,1,2,1,2,3]))
result.numpy()

array([[-2.1744519e-05,  3.8744774e-02,  4.5454454e-02],
       [ 3.7292365e-02, -4.5226324e-02, -2.5365019e-02],
       [-5.4663531e-03,  4.1066814e-02, -3.5211097e-02],
       [ 3.7292365e-02, -4.5226324e-02, -2.5365019e-02],
       [-5.4663531e-03,  4.1066814e-02, -3.5211097e-02],
       [-3.7309039e-02,  3.7818067e-03,  1.9867215e-02]], dtype=float32)

In [ ]:
first_sentence

array([5, 3, 2, 4, 0, 0], dtype=int32)

In [ ]:
len(first_sentence)

6

In [ ]:
# embedding for 1 sentence
result=embedding_layer(first_sentence)
print(result.shape)
result.numpy()

(6, 3)


array([[-3.1944454e-02, -2.0747567e-02, -4.5926087e-03],
       [-3.7309039e-02,  3.7818067e-03,  1.9867215e-02],
       [-5.4663531e-03,  4.1066814e-02, -3.5211097e-02],
       [-8.1157684e-03,  1.8775824e-02,  1.3189439e-02],
       [-2.1744519e-05,  3.8744774e-02,  4.5454454e-02],
       [-2.1744519e-05,  3.8744774e-02,  4.5454454e-02]], dtype=float32)

**Question**: Can you make sense why the shape of our output is 4, 6, 3 ?

In [ ]:
# embedding for every sentence
result=embedding_layer(padded)
print(result.shape)
result.numpy()

(4, 6, 3)


array([[[-3.1944454e-02, -2.0747567e-02, -4.5926087e-03],
        [-3.7309039e-02,  3.7818067e-03,  1.9867215e-02],
        [-5.4663531e-03,  4.1066814e-02, -3.5211097e-02],
        [-8.1157684e-03,  1.8775824e-02,  1.3189439e-02],
        [-2.1744519e-05,  3.8744774e-02,  4.5454454e-02],
        [-2.1744519e-05,  3.8744774e-02,  4.5454454e-02]],

       [[-3.1944454e-02, -2.0747567e-02, -4.5926087e-03],
        [-3.7309039e-02,  3.7818067e-03,  1.9867215e-02],
        [-5.4663531e-03,  4.1066814e-02, -3.5211097e-02],
        [ 3.3895124e-02,  1.9534323e-02,  2.2506867e-02],
        [-2.1744519e-05,  3.8744774e-02,  4.5454454e-02],
        [-2.1744519e-05,  3.8744774e-02,  4.5454454e-02]],

       [[ 3.6478329e-02,  1.1744276e-03, -7.2725639e-03],
        [-3.7309039e-02,  3.7818067e-03,  1.9867215e-02],
        [-5.4663531e-03,  4.1066814e-02, -3.5211097e-02],
        [-8.1157684e-03,  1.8775824e-02,  1.3189439e-02],
        [-2.1744519e-05,  3.8744774e-02,  4.5454454e-02],
        [-

Now, you have successfuly learn how to convert the sentences of text into numeric tokens and for each numeric token, you know how to retrieve the corresponding word embedding vector!

Now time to learn how to build a neural network to fetch a real IMDB dataset for it to train and update these values in embedding layers!

# Stage 2: IMBD Sentiment Analysis with simple neural network model

## Prepare the data

Same old libraries like above ;)

In [ ]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

By now, you can start to make sense of these parameters we set at the beginning. Feel free to tune your model by changing these parameters!

In [ ]:
vocab_size = 10000
embedding_dim = 100
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

Let's import tensorflow datasets library and download IDMB text reviews dataset!

Here are more info of IMDB dataset we got from Tensorflow: https://www.tensorflow.org/datasets/catalog/imdb_reviews

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [ ]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
len(training_sentences)

In [ ]:
print("Sentence:", training_sentences[0])
print("Label:", training_labels[0])
print("---------------------------------")
print("Sentence:", training_sentences[1])
print("Label:", training_labels[1])
print("---------------------------------")
print("Sentence:", training_sentences[3])
print("Label:", training_labels[3])
print("---------------------------------")
print("Sentence:", training_sentences[5])
print("Label:", training_labels[5])

Time to tokenize our sentences and pad them!

In [ ]:
# For Training Data:

# create a tokenizer with num_words and oov_token attributes
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# use that tokenizer to fit the training sentences we got above
tokenizer.fit_on_texts(training_sentences)

# retrieve the word_index back from the tokenizer
word_index = tokenizer.word_index

# use the tokenizer we have fitted on the training sentences and create encoded sequences of index of training setences
sequences = tokenizer.texts_to_sequences(training_sentences)

# remember to pad the sequences for them to be on the same length with max_len and truncating attributes
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)


# For Testing Data: 

# we only use the same tokenize we got above and create encoded sequences of index of test setences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

# remember to pad the sequences for them to be on the same length with max_len and truncating attributes
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

If you implements the above code correctly, this decode should give you back the original text from encoded sequences.

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print("Original:", training_sentences[0])
print("Tokenize:", sequences[0])
print("Reduce or Padded:", padded[0])
print("Decode:", decode_review(padded[0]))
print("Label:", training_labels_final[0])

It's always good to visualize training and validation loss or accuracy after training the model!

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

## Embedding layer with Flatten Layer


The process of training the model have repeated 4 steps:

1. Forward Propagation (Plug in the word tokens of sentences and output one number for sentiment result - output)

2. Compare the output to the expected output (label) by calculating the loss function (measuring how bad is your output of your model)

3. Back Propagation (Calculate how much you need to update your parameter values in your model so the next step you do the forward propagation will be slightly better)

4. Gradient Descent (Actually take the values from step 3 and update those parameters in the model)

After training the model for many epoches, the model will have good parameters to achieve high accuracy and low loss! :)

So define the model as below to tell the model how to do in **step 1** (forward propagation).

Next loss function "binary_crossentropy" is loss function to measure how bad the output compared to the expected output in **step 2**.

Optimizer is the function is to define how you should update the parameters in **step 4**.


In [ ]:
# YOUR CODE
# Build model with Flatten(), a Fully Connected Layer like 8 neurons and the last layer is a Fully Connected Layer with 1 neuron with activation is 'sigmoid'

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # your code
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


To train the model, you can simply call the method fit ;) 

In [ ]:
num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Plot how well your model doing after training!

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

I won't discuss about overfitting and underfitting in this workshop because that topic alone can take another workshop to do :(

### Prediction sentiment of some sentences

In [ ]:
# YOU CAN TRY YOUR OWN EXAMPLES
# Hopefully, the first and third sentences are positive and middle one is negative

sentence = ["I really think this is amazing. honest.", "It sucks and so bad"]
sequences = tokenizer.texts_to_sequences(sentence)
print("Sequences:", sequences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print("Padded:", padded)
print("Prediction:", model.predict(padded))

### [Fun] Visualize your embedding vectors in 3 dimension space

You need to export the Embedding layer into vecs and meta files to visualise 

Run this code to export the values of vectors in embedding (vecs.tsv) and coressponding words (meta.tsv). Thus, you will have two files in total.

Remember to click "Allow to download multiple files" on Chrome to download two files at the same time!

Open http://projector.tensorflow.org/ and load those two files you just download so see the visualization in 3D or 2D of your word embedding!

In [ ]:
import io
def export_embedding_tsv(model):
  e = model.layers[0]
  weights = e.get_weights()[0]
  #print(weights.shape) # shape: (vocab_size, embedding_dim)

  out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
  out_m = io.open('meta.tsv', 'w', encoding='utf-8')
  for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
  out_v.close()
  out_m.close()

  try:
    from google.colab import files
  except ImportError:
    pass
  else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

In [ ]:
export_embedding_tsv(model)

## Extra Models to play around with (Optional)

### Embedding layer with Global Average Pooling Layer



![](https://i.imgur.com/H0Gh7wA.png)

In [ ]:
# YOUR CODE
# Use Global Average Pooling 1D and 2 Dense Layers with the last layer is one neuron and activation is sigmoid. 

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # your code
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


In [ ]:
num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

Do you notice anything different from the last model about speed and accuracy?

### Embedding layer with a LSTM layer

In [ ]:
# YOUR CODE
# Use LSTM layer with 2 Dense Layers followed it

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # your code
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


In [ ]:
num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

### Embedding layer with a Bidirectional LSTM layer



In [ ]:
# YOUR CODE
# Use bidirectional LSTM with 2 dense layers

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # your code
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


In [ ]:
num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

In [ ]:
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

### Embedding layer with multiple Bidirectional LSTM layers



In [ ]:
# YOUR CODE
# Use stacked bidirectional LSTM (2 bidirectional LSTM stacked on each other) with two dense layers 
# be mindful about return_sequence 

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # your code
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 10
history = model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")